## RAG pipeline using Typesense

In [1]:
import typesense

In [2]:
client = typesense.Client({
    'nodes': [{
        'host': '92pg7uq8oxmi04sfp-1.a1.typesense.net',
        'port': '443',
        'protocol': 'https'
    }],
    'api_key': 'Xfzt4S6IVQKsnMyIogxZ6KS94LR1oZUx',
    'connection_timeout_seconds': 2
})


"""books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string' },
    {'name': 'authors', 'type': 'string[]', 'facet': True },

    {'name': 'publication_year', 'type': 'int32', 'facet': True },
    {'name': 'ratings_count', 'type': 'int32' },
    {'name': 'average_rating', 'type': 'float' }
  ],
  'default_sorting_field': 'ratings_count'
}
print(client.collections.create(books_schema))"""

"books_schema = {\n  'name': 'books',\n  'fields': [\n    {'name': 'title', 'type': 'string' },\n    {'name': 'authors', 'type': 'string[]', 'facet': True },\n\n    {'name': 'publication_year', 'type': 'int32', 'facet': True },\n    {'name': 'ratings_count', 'type': 'int32' },\n    {'name': 'average_rating', 'type': 'float' }\n  ],\n  'default_sorting_field': 'ratings_count'\n}\nprint(client.collections.create(books_schema))"

In [3]:
client

In [4]:
with open('C:/Users/19736/Desktop/Project/Yt_rag/books.jsonl', 'r', encoding='utf-8') as jsonl_file:
    data = jsonl_file.read()
    client.collections['books'].documents.import_(data)

In [5]:
search_parameters = {
    'q': "harry potter",
    'query_by': "title,authors",
    'sort_by': "ratings_count:desc",
    'filter_by': "publication_year:<1998"
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name

In [6]:
### Langcahin + Typsense + Groq LLM + RAG Appplication

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\19736\Desktop\Project\Yt_rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
genai_api_key = os.getenv("GOOGLE_GENERATIVE_AI_API_KEY")
typesense_api_key = os.getenv("TYPESENSE_API_KEY")

In [8]:
typesense_api_key

'Xfzt4S6IVQKsnMyIogxZ6KS94LR1oZUx'

In [9]:
loader = TextLoader('C:/Users/19736/Desktop/Project/Yt_rag/test.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()


C:\Users\19736\AppData\Local\Temp\ipykernel_68648\2262608678.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\19736\AppData\Local\Temp\ipykernel_68648\2262608678.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [10]:
docsearch = Typesense.from_documents(
    docs, 
    embeddings,
    typesense_client_params={
        'host': '92pg7uq8oxmi04sfp-1.a1.typesense.net',
        'port': '443',
        'protocol': 'https',
        'typesense_api_key': typesense_api_key,
        'typesense_collection_name': "lang-chain"
  },
)

In [11]:
query ="what is Artificial intelligence?"
found_docs = docsearch.similarity_search(query)
print(found_docs[0].page_content)

Artificial intelligence (AI) is the development of computer systems that can perform tasks requiring human intelligence, such as learning, reasoning, and problem-solving, by processing data and adapting over time. AI benefits include automation of repetitive tasks, improved accuracy, and enhanced decision-making across various fields like healthcare, finance, and manufacturing. Common examples include virtual assistants, chatbots, and systems used for fraud detection and medical image analysis. 
Key concepts and benefits
How it works: Instead of relying solely on explicit instructions, AI systems learn from large amounts of data to find patterns and make predictions.
Core technologies: Major branches of AI include machine learning, deep learning, and natural language processing (NLP).
Benefits: AI can automate workflows, making businesses more efficient and productive. It can also provide deeper insights from complex data, leading to more informed decisions.
Everyday examples: You inte

In [12]:
retriever = docsearch.as_retriever()
retriever

VectorStoreRetriever(tags=['Typesense', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.typesense.Typesense object at 0x0000023937760590>, search_kwargs={})

In [13]:
query = "AI indepth explanation"
print(retriever.invoke(query)[0].page_content)

Artificial intelligence (AI) is the development of computer systems that can perform tasks requiring human intelligence, such as learning, reasoning, and problem-solving, by processing data and adapting over time. AI benefits include automation of repetitive tasks, improved accuracy, and enhanced decision-making across various fields like healthcare, finance, and manufacturing. Common examples include virtual assistants, chatbots, and systems used for fraud detection and medical image analysis. 
Key concepts and benefits
How it works: Instead of relying solely on explicit instructions, AI systems learn from large amounts of data to find patterns and make predictions.
Core technologies: Major branches of AI include machine learning, deep learning, and natural language processing (NLP).
Benefits: AI can automate workflows, making businesses more efficient and productive. It can also provide deeper insights from complex data, leading to more informed decisions.
Everyday examples: You inte